<h1 style="text-align: center; font-family: Verdana; font-size: 32px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; font-variant: small-caps; letter-spacing: 3px; color: #74d5dd; background-color: #ffffff;">Human Protein Atlas - Single Cell Classification</h1>
<h2 style="text-align: center; font-family: Verdana; font-size: 22px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">Integrated Gradients For Inference</h2>
<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>

---

<b style="text-transform: uppercase;"><center>Can we use Integrated Gradients to tell which cells are which within an image containing multiple labels?</center></b>

<h2 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;">TABLE OF CONTENTS</h2>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#imports">0&nbsp;&nbsp;&nbsp;&nbsp;IMPORTS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#xai_background">1&nbsp;&nbsp;&nbsp;&nbsp;XAI BACKGROUND INFORMATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#ig_background">2&nbsp;&nbsp;&nbsp;&nbsp;IG BACKGROUND INFORMATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#setup">3&nbsp;&nbsp;&nbsp;&nbsp;SETUP</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">4&nbsp;&nbsp;&nbsp;&nbsp;HELPER FUNCTIONS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#model">5&nbsp;&nbsp;&nbsp;&nbsp;LOAD AND DEMONSTRATE MODEL PREDICTION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#calc_ig">6&nbsp;&nbsp;&nbsp;&nbsp;CALCULATE INTEGRATED GRADIENTS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#viz_ig">7&nbsp;&nbsp;&nbsp;&nbsp;VISUALIZE INTEGRATED GRADIENTS</a></h3>

---

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;" id="imports">0&nbsp;&nbsp;IMPORTS</a>

In [ ]:
print("\n... IMPORTS STARTING ...\n")
print("\n\tVERSION INFORMATION")

# Machine Learning and Data Science Imports
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import scipy; print(f"\t\t– SCIPY VERSION: {scipy.__version__}");

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
import multiprocessing
from glob import glob
import warnings
import requests
import imageio
import IPython
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import tqdm
import time
import gzip
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2

# PRESETS
LBL_NAMES = ["Nucleoplasm", "Nuclear Membrane", "Nucleoli", "Nucleoli Fibrillar Center", "Nuclear Speckles", "Nuclear Bodies", "Endoplasmic Reticulum", "Golgi Apparatus", "Intermediate Filaments", "Actin Filaments", "Microtubules", "Mitotic Spindle", "Centrosome", "Plasma Membrane", "Mitochondria", "Aggresome", "Cytosol", "Vesicles", "Negative"]
INT_2_STR = {x:LBL_NAMES[x] for x in np.arange(19)}
INT_2_STR_LOWER = {k:v.lower().replace(" ", "_") for k,v in INT_2_STR.items()}
STR_2_INT_LOWER = {v:k for k,v in INT_2_STR_LOWER.items()}
STR_2_INT = {v:k for k,v in INT_2_STR.items()}
FIG_FONT = dict(family="Helvetica, Arial", size=14, color="#7f7f7f")
LABEL_COLORS = [px.colors.label_rgb(px.colors.convert_to_RGB_255(x)) for x in sns.color_palette("Spectral", len(LBL_NAMES))]
LABEL_COL_MAP = {str(i):x for i,x in enumerate(LABEL_COLORS)}

CMAP_DICT = {
    0:sns.color_palette("rocket", as_cmap=True),
    1:sns.color_palette("mako", as_cmap=True),
    2:sns.color_palette("marma", as_cmap=True),
    3:sns.color_palette("viridis", as_cmap=True),
    4:"gray",
}

print("\n\n... IMPORTS COMPLETE ...\n")

print("\n\n... TPU SETUP ...\n")

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
except ValueError: # no TPU found, detect GPUs
    strategy = tf.distribute.get_strategy() # for GPU or multi-GPU machines
    print("\n... NOT USING TPU ...\n")
    
N_REPLICAS = strategy.num_replicas_in_sync
print(f"... Number Of Accelerators: {N_REPLICAS} ...\n")

print("\n... TPU SETUP COMPLETE ...\n")

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="xai_background">1&nbsp;&nbsp;XAI BACKGROUND INFORMATION</a>

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.1  WHAT IS EXPLAINABLE AI - GENERAL INFO</h3>

---

For the purposes of this notebook and my explanation, I will be logically seperating explainable AI into two seperate branches. 

[**See this excerpt/paper that explains the branches in more detail.**](https://arxiv.org/pdf/1907.07374.pdf)

> "The two major categories presented here, namely perceptive interpretability and interpretability by mathematical structures, appear to present different polarities within the notion of interpretability. 
> 
> As an example for the difficulty with perceptive interpretability, when a visual evidence is given erroneously, the underlying mathematical structure may not seem to provide useful clues on the mistakes. 
> 
> On the other hand, a mathematical analysis of patterns may provide
information in high dimensions. They can only be easily perceived once the pattern is brought into lower dimensions, abstracting some fine-grained information we could not yet prove is not discriminative with measurable certainty."

[**<sup><sub>Tjoa, E., & Guan, C. (2019). A survey on explainable artificial intelligence (XAI): Towards
medical XAI. arXiv preprint arXiv:1907.0737</sub></sup>**](https://arxiv.org/pdf/1907.07374.pdf)
    
<br><br>

<b style="text-decoration: underline; font-family: Verdana;">PERCEPTIVE</b>
    
In short this is interpretability that can be observed by humans. Often the explanations arising through this branch are obvious to humans or already known.


<br>

<b style="text-decoration: underline; font-family: Verdana;">MATHEMATICAL</b>
    
In short this is interpretability that can only be observed by first applying mathematical manipulations to the data. An example technique that most are familiar with is clustering [**`(t-SNE)`**](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)


<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.2  THE GOAL</h3>

---

**Pull the veil back on black-box machine learning models and help users understand how/why a model makes the decisions that it does. This can inform on how to improve the model as well as being useful for identifying things like bias and overfitting**

<img src="https://i.ibb.co/ZXdBQ4D/Screen-Shot-2020-07-07-at-10-24-16-AM.png">


<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.3  CURRENT XAI APPROACHES</h3>

---

| Algorithm                     	| Type         	| Description                                                                                                                                                                                                                                                                                                                                                                                                  	|
|:-------------------------------	|:--------------	|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	|
| Integrated Gradients        	| Gradient     	| Approximates the integral of gradients along the path (straight line from baseline to input) sand multiplies with (input - baseline)                                                                                                                                                                                                                                                                         	|
| DeepLift                    	| Application  	| Explains differences in the non-linear activations' outputs in terms of the differences of the input from its corresponding reference.                                                                                                                                                                                                   	|
| DeepLiftSHAP                	| Gradient     	| An extension of DeepLift that approximates SHAP values.<br>For each input example it considers a distribution of baselines and computes the expected value of the attributions based on DeepLift algorithm across all input-baseline pairs.                                                                 	|
| GradientSHAP                	| Gradient     	| Approximates SHAP values based on the expected gradients.<br>It adds gaussian noise to each input example #samples times, selects a random point between each sample and randomly drawn baseline from baselines' distribution, computes the gradient for it and multiples it with (input - baseline).<br>Final SHAP values represent the expected values of gradients * (input - baseline) for each input example. 	|
| Input * Gradient              	| Gradient     	| Multiplies model inputs with the gradients of the model outputs w.r.t. those inputs.                                                                                                                                                                                                                                                                                                                         	|
| Saliency                     	| Gradient     	| The gradients of the output w.r.t. inputs.                                                                                                                                                                                                                                                                                                                                                                   	|
| Guided BackProp / DeconvNet 	| Gradient     	| Computes the gradients of the model outputs w.r.t. its inputs.<br>If there are any RELUs present in the model, their gradients will be overridden so that only positive gradients of the inputs (in case of Guided BackProp) and outputs (in case of deconvnet) are back-propagated.                                                                                                                            	|
| Guided GradCam                	| Gradient     	| Computes the element-wise product of Guided BackProp and up-sampled positive GradCam attributions.                                                                                                                                                                                                                                                                                                           	|
| LayerGradCam                  	| Gradient     	| Computes the gradients of model outputs w.r.t. selected input layer, averages them for each output channel and multiplies with the layer activations.                                                                                                                                                                                                                                                        	|
| Layer Internal Influence      	| Gradient     	| Approximates the integral of gradients along the path from baseline to inputs for selected input layer.                                                                                                                                                                                                                                                                                                      	|
| Layer Conductance            	| Gradient     	| Decomposes integrated gradients via chain rule.<br>It approximates the integral of gradients defined by a chain rule, described as the gradients of the output w.r.t. to the neurons multiplied by the gradients of the neurons w.r.t. the inputs, along the path from baseline to inputs.<br>Finally, the latter is multiplied by (input - baseline).                                                             	|
| Layer Gradient * Activation   	| Gradient     	| Computes element-wise product of layer activations and the gradient of the output w.r.t. that layer.                                                                                                                                                                                                                                                                                                         	|
| Layer Activation              	| -            	| Computes the inputs or outputs of selected layer.                                                                                                                                                                                                                                                                                                                                                            	|
| Feature Ablation            	| Perturbation 	| Assigns an importance score to each input feature based on the magnitude changes in model output or loss when those features are replaced by a baseline (usually zeros) based on an input feature mask.                                                                                                                                                                                                      	|
| Feature Permutation           	| Perturbation 	| Assigns an importance score to each input feature based on the magnitude changes in model output or loss when those features are permuted based on input feature mask.                                                                                                                                                                                                                                       	|
| Occlusion                     	| Perturbation 	| Assigns an importance score to each input feature based on the magnitude changes in model output when those features are replaced by a baseline (usually zeros) using rectangular sliding windows and sliding strides.<br>If a features is located in multiple hyper-rectangles the importance scores are averaged across those hyper-rectangles.                                                               	|
| Shapely Value                 	| Perturbation 	| Computes feature importances based on all permutations of all input features.<br>It adds each feature for each permutation one-by-one to the baseline and computes the magnitudes of output changes for each feature which are ultimately being averaged across all permutations to estimate final attribution score.                                                                                           	|
| Shapely Value Sampling        	| Perturbation 	| Similar to Shapely value, but instead of considering all feature permutations it considers only #samples random permutations.                                                                                                                                                                                                                                                                                	|
| NoiseTunnel                   	| -            	| Depends on the choice of above mentioned attribution algorithm                                                                                                                                                                                                                                                                                                                                                                        	|


<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.4  WHERE IS XAI NEEDED?</h3>

---

***Obviously things like the weakly-supervised tasks in this competition may require XAI***

XAI can be used for a wide range of things that we won't get into here (protecting against bias, protecting against overfitting, detecting features, etc.)

One other place XAI can be used is when working with black-box models. To understand what that is we will see the definitions and examples of the terms: **Transparent and Black-Box Models**:

<br>

<b style="text-decoration: underline; font-family: Verdana;">TRANSPARENT MODELS</b>

These are models/algorithms that are easily interpretable and **DO NOT** (generally) requre XAI. 

*Although occasionally post-hoc analysis is required or basic explainability tools.*

* Linear/Logistic Regression
* Decision Trees
* K-Nearest Neighbors
* Rule Based Learners
* General Additive Models
* Bayesian Models
    
<br>

<b style="text-decoration: underline; font-family: Verdana;">BLACK-BOX MODELS</b>

These are models/algorithms that are NOT easily interpretable and **DO** requre XAI. 

*This is not an exhaustive list of black-box models. It is simply the more common black-box models.*

* Tree Ensembles
* Support Vector Machines
* Multi-Layer Neural Network (MLPNN)
* Convolutional Neural Network (CNN)
* Recurrent Neural Network (RNN)


<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: uppercase; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.4  Why Is XAI Needed: The Case For Growing Global AI Regulation</h3>

---

Many regulatory bodies have begun to encourage or enforce explainability in predictive algorithms used in the public domain.<br><br>**See below!**<br><sub>*(list was created roughly a year ago)*</sub>

- GDPR: Article 22 empowers individuals with the right to demand an explanation of how an
automated system made a decision that affects them.
- Algorithmic Accountability Act 2019: Requires companies to provide an assessment of the risks posed by
the automated decision system to the privacy or security and the risks that contribute to inaccurate, unfair,
biased, or discriminatory decisions impacting consumers
- California Consumer Privacy Act: Requires companies to rethink their approach to capturing,
storing, and sharing personal data to align with the new requirements by January 1, 2020.
- Washington Bill 1655: Establishes guidelines for the use of automated decision systems to protect
consumers, improve transparency, and create more market predictability.
- Massachusetts Bill H.2701: Establishes a commission on automated decision-making,
transparency, fairness, and individual rights.
- Illinois House Bill 3415: States predictive data analytics determining creditworthiness or hiring
decisions may not include information that correlates with the applicant race or zip code.

<a style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="ig_background">2&nbsp;&nbsp;INTEGRATED GRADIENTS BACKGROUND INFORMATION</a>

**This notebook will show how to implement Integrated Gradients (IG) for this competition.**

IG is an Explainable AI (XAI) technique introduced in the paper **[Axiomatic Attribution for Deep Networks](https://arxiv.org/abs/1703.01365)**

---

***LINKS***<br>
<sub>&nbsp;&nbsp;&nbsp;&nbsp;- [**Tensorflow - Google Colab This Is Heavily Based Off Of**](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/interpretability/integrated_gradients.ipynb)</sub><br>

---

**I**ntegrated **G**radients (**IG**) aims to explain the relationship between a model's predictions in terms of its features. It has many use cases including understanding feature importances, identifying data skew, and debugging model performance.

**IG** has become a popular interpretability technique due to its broad applicability to any differentiable model (e.g. images, text, structured data), ease of implementation, theoretical justifications, and computational efficiency relative to alternative approaches that allows it to scale to large networks and feature spaces such as images.

Go to this notebook to see the implementation of IG. In it, we will walk through an implementation of **IG** step-by-step to understand the pixel feature importances of an image classifier. 

---

As an example, consider this **[image](https://commons.wikimedia.org/wiki/File:San_Francisco_fireboat_showing_off.jpg)** of a fireboat spraying jets of water. 

You would classify this image as a **fireboat** and might highlight the pixels making up the **boat** and **water cannons** as being important to your decision. 

The model will also classify this image as a fireboat later on in this tutorial; however, does it highlight the same pixels as important when explaining its decision?

In the images below titled "**IG** Attribution Mask" and "Original + **IG** Mask Overlay" you can see that the model instead highlights (in purple) the pixels comprising the boat's **water cannons** and **jets of water** as being ***more important than the boat itself*** to its decision. 

How will the model generalize to new fireboats? What about fireboats without water jets? 

Read on to learn more about how **IG** works and how to apply **IG** to models to better understand the relationship between their predictions and underlying features.

![IG Example](https://www.tensorflow.org/tutorials/interpretability/images/IG_fireboat.png)

<a style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="setup">3&nbsp;&nbsp;NOTEBOOK SETUP</a>

In [ ]:
# Define the root data directory
TRAIN_IMG_DIR = KaggleDatasets().get_gcs_path("hpa-512512")
LOCAL_DATA_DIR = "/kaggle/input/hpa-single-cell-image-classification"

# Capture all the relevant full image paths
TRAIN_IMG_PATHS = tf.io.gfile.glob(os.path.join(TRAIN_IMG_DIR, '*.png'))
MODEL_DIR = "/kaggle/input/hpa-xai-ig-tfrecords-tpu-training/model"

# Capture all the relevant full image paths
TRAIN_IMG_PATHS = tf.io.gfile.glob(os.path.join(TRAIN_IMG_DIR, '*.png'))

print(f"\n... Recall that 4 training images compose one example (R,G,B,Y) ...")
print(f"... \t– i.e. The first 4 training files are:")
for path in [x.rsplit('/',1)[1] for x in TRAIN_IMG_PATHS[:4]]: print(f"... \t\t– {path}")
print(f"\n... The number of training images is {len(TRAIN_IMG_PATHS)} i.e. {len(TRAIN_IMG_PATHS)//4} 4-channel images ...")

# Define paths to the relevant csv files &
# create the relevant dataframe objects
TRAIN_CSV = os.path.join(LOCAL_DATA_DIR, "train.csv")
train_df = pd.read_csv(TRAIN_CSV)
print("\n\nTRAIN DATAFRAME\n\n")
display(train_df.head(3))

SS_CSV = os.path.join(LOCAL_DATA_DIR, "sample_submission.csv")
ss_df = pd.read_csv(SS_CSV)
print("\n\nSAMPLE SUBMISSION DATAFRAME\n\n")
display(ss_df.head(3))

# Create Various-Label-Count Dataframes
dfs_by_label_cnt = {(i+1):train_df[train_df.Label.str.count("\|")==i].reset_index(drop=True) for i in range(5)}
display(dfs_by_label_cnt[1].head(3))
display(dfs_by_label_cnt[2].head(3))
display(dfs_by_label_cnt[3].head(3))
display(dfs_by_label_cnt[4].head(3))
display(dfs_by_label_cnt[5].head(3))

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="helper_functions">4&nbsp;&nbsp;HELPER FUNCTIONS</a>

In [ ]:
def decode_img(image_data, resize_to=(512,512)):
    image = tf.image.decode_png(image_data, channels=1)
    # explicit size needed for TPU
    image = tf.reshape(image, resize_to) 
    return tf.cast(image, tf.float32)


def load_image(img_id, img_dir, resize_to=(512,512), tpu_style=False):
    """ Load An Image Using ID and Directory Path - Composes 4 Individual Images """
    if not tpu_style:
        rgby = [
            np.asarray(Image.open(os.path.join(img_dir, img_id+f"_{c}.png")).resize(resize_to), np.uint8) \
            for c in ["red", "green", "blue", "yellow"]
        ]
        return np.stack(rgby, axis=-1)
    else:
        rgby = [
            decode_img(tf.io.read_file(os.path.join(img_dir, img_id+f"_{c}.png")), resize_to) \
            for c in ["red", "green", "blue", "yellow"]
        ]
        return tf.stack(rgby, axis=-1)

def plot_rgb(arr, figsize=(12,12)):
    """ Plot 3 Channel Microscopy Image """
    plt.figure(figsize=figsize)
    plt.title(f"RGB Composite Image", fontweight="bold")
    plt.imshow(arr)
    plt.axis(False)
    plt.show()
    
    
def convert_rgby_to_rgb(arr, boost_green=False):
    """ Convert a 4 channel (RGBY) image to a 3 channel RGB image.
    
    Advice From Competition Host/User: lnhtrang

    For annotation (by experts) and for the model, I guess we agree that individual 
    channels with full range px values are better. 
    In annotation, we toggled the channels. 
    For visualization purpose only, you can try blending the channels. 
    For example, 
        - red = red + yellow
        - green = green + yellow/2
        - blue=blue.
        
    Args:
        arr (numpy array): The RGBY, 4 channel numpy array for a given image
        boost_green (numpy array): Whether to boost the intensity of the green channel
    
    Returns:
        RGB Image
    """
    
    rgb_arr = np.zeros_like(arr[..., :-1])
    if boost_green:
        rgb_arr[..., 0] = arr[..., 0]/1.25
        rgb_arr[..., 1] = np.clip(arr[..., 1]*2+arr[..., 3]/5, 0, 255)
        rgb_arr[..., 2] = arr[..., 2]/1.25
        rgb_arr = rgb_arr.astype(np.uint8)
    else:
        rgb_arr[..., 0] = arr[..., 0]
        rgb_arr[..., 1] = arr[..., 1]+arr[..., 3]/2
        rgb_arr[..., 2] = arr[..., 2]
    
    return rgb_arr



def plot_ex(arr, figsize=(20,6), title=None, plot_merged=True, rgb_only=False):
    """ Plot 4 Channels Side by Side """
    if plot_merged and not rgb_only:
        n_images=5 
    elif plot_merged and rgb_only:
        n_images=4
    elif plot_merged and rgb_only:
        n_images=4
    else:
        n_images=3
    plt.figure(figsize=figsize)
    if type(title) == str:
        plt.suptitle(title, fontsize=20, fontweight="bold")

    for i, c in enumerate(["Red Channel – Microtubles", "Green Channel – Protein of Interest", "Blue - Nucleus", "Yellow – Endoplasmic Reticulum"]):
        if not rgb_only:
            ch_arr = np.zeros_like(arr[..., :-1])        
        else:
            ch_arr = np.zeros_like(arr)
        if c in ["Red Channel – Microtubles", "Green Channel – Protein of Interest", "Blue - Nucleus"]:
            ch_arr[..., i] = arr[..., i]
        else:
            if rgb_only:
                continue
            ch_arr[..., 0] = arr[..., i]
            ch_arr[..., 1] = arr[..., i]
        plt.subplot(1,n_images,i+1)
        plt.title(f"{c.title()}", fontweight="bold")
        plt.imshow(ch_arr)
        plt.axis(False)
        
    if plot_merged:
        plt.subplot(1,n_images,n_images)
        
        if rgb_only:
            plt.title(f"Merged RGB", fontweight="bold")
            plt.imshow(arr)
        else:
            plt.title(f"Merged RGBY into RGB", fontweight="bold")
            plt.imshow(convert_rgby_to_rgb(arr))
        plt.axis(False)
        
    plt.tight_layout(rect=[0, 0.2, 1, 0.97])
    plt.show()
    
    
def flatten_list_of_lists(l_o_l):
    return [item for sublist in l_o_l for item in sublist]


def load_batch_of_images(df, id_list, img_dir, resize_to=(512,512), return_labels=True, tpu_style=False):
    if not return_labels:
        return np.stack([load_image(ID, img_dir, resize_to) for ID in id_list], axis=0)
    else:
        lbls = df[df.ID.isin(id_list)].Label.apply(lambda x: [int(l) for l in x.split("|")]).to_list()
        return np.stack([load_image(ID, img_dir, resize_to, tpu_style=tpu_style) for ID in id_list], axis=0), lbls
    
def plot_batch_of_images(img_batch, lbl_batch=None, pred_batch=None, n_cols=4, labels_as_strs=True, boost_green=True):
    n_imgs = img_batch.shape[0]
    if not lbl_batch:
        lbl_batch = [None,]*n_imgs
    if not pred_batch:
        pred_batch = [None,]*n_imgs
    
    plt.figure(figsize=(19, int(5.5*np.ceil(n_imgs/n_cols))))
    for i, (img, lbl, pred) in enumerate(zip(img_batch, lbl_batch, pred_batch)):
        plt.subplot(int(np.ceil(n_imgs/n_cols)), n_cols, i+1)
        if lbl or pred:
            title_str = ""
            if lbl:
                if labels_as_strs:
                    title_str+=f"GT LABEL: {[INT_2_STR[l] for l in lbl]}"
                else:
                    title_str+=f"GT LABEL: {sorted(list(lbl))}"
            if pred:
                if labels_as_strs:
                    title_str+=f"\nPRED LABEL: {[INT_2_STR[p] for p in pred[0]]}"
                else:
                    title_str+=f"\nPRED LABEL: {sorted(list(pred[0]))}"
            plt.title(title_str.strip("\n"), fontweight="bold")
        plt.imshow(convert_rgby_to_rgb(img, boost_green=boost_green))
        plt.axis(False)

    plt.tight_layout()
    plt.show()
    
def get_pred(model, img_batch, conf_thresh=0.3, drop_yellow=True):
    if drop_yellow:
        img_batch = img_batch[..., :-1]
    pred_batch = model.predict(img_batch)
    return [np.where(p>conf_thresh) for p in pred_batch]

def interpolate_images(baseline, image, alphas=None):
    if alphas is None:
        alphas = tf.linspace(start=0.0, stop=1.0, num=25)

    # type coercion
    _image = tf.cast(image, alphas.dtype)
    _baseline = tf.cast(baseline, alphas.dtype)
    
    if tf.math.reduce_max(_image)>1.:
        _image /= 255.
        
    if tf.math.reduce_max(_baseline)>1.:
        _baseline /= 255.
    
    # Give alphas, baseline, and input all 4 dimensions (b, w, h, c)
    alphas_x = alphas[:, tf.newaxis, tf.newaxis, tf.newaxis]
    baseline_x = tf.expand_dims(_baseline, axis=0)
    
    input_x = tf.expand_dims(_image, axis=0)
    
    # Calculate delta
    delta = input_x - baseline_x

    # Create the stepwise images between baseline and the original image
    # As alphas_x increases the contribution of the original image 
    # (represented by delta in our case) grows to become the original value
    #
    # This is in essence, stepping from pure black towards the original
    images = baseline_x + alphas_x*delta
    return images*255

def compute_gradients(model, images, target_class_idx):
    if images.dtype != tf.float32:
        images = tf.cast(images, tf.float32)
    
    if tf.math.reduce_max(images)<=1.:
        images *= 255.
        
    with tf.GradientTape() as tape:
        tape.watch(images)
        target_pred = model(images)[:, target_class_idx]
        
    return tape.gradient(target_pred, images)

def integral_approximation(gradients):
    # riemann_trapezoidal
    grads = (gradients[:-1] + gradients[1:]) / tf.constant(2.0)
    integrated_gradients = tf.math.reduce_mean(grads, axis=0)
    return integrated_gradients

def process_attributions(attributions, rescale_power=1, grayscale=False):
    def _min_max_tensor(tensor, exp=1):
        return tf.math.divide(tf.subtract(tensor, tf.reduce_min(tensor)), 
                               tf.subtract(tf.reduce_max(tensor), tf.reduce_min(tensor)))**exp
    
    pos_attribution_mask = \
        tf.reduce_sum(tf.math.abs(tf.clip_by_value(attributions, 0, 10000)), axis=-1)
    neg_attribution_mask = \
        tf.reduce_sum(tf.math.abs(tf.clip_by_value(attributions, -10000, 0)), axis=-1)

    pos_attribution_mask = _min_max_tensor(pos_attribution_mask, 1/rescale_power)
    neg_attribution_mask = _min_max_tensor(neg_attribution_mask, 1/rescale_power)
    
    if grayscale:
        return tf.clip_by_value(pos_attribution_mask+neg_attribution_mask, 0.0, 1.0)
    else:
        return pos_attribution_mask, neg_attribution_mask

def make_attribution_cmap():
    # Make special colour maps
    pos_cmap = sns.dark_palette("red")
    pos_cmap.insert(0, (0.,0.,0.))
    pos_cmap = ListedColormap(pos_cmap)

    neg_cmap = sns.dark_palette("blue")
    neg_cmap.insert(0, (0.,0.,0.))
    neg_cmap = ListedColormap(neg_cmap)

    return pos_cmap, neg_cmap

def plot_img_attributions(attributions, baseline, image, class_label, overlay_alpha=0.5, save_fig=True, rescale_power=1):        
    pos_cmap, neg_cmap = make_attribution_cmap()

    # Sum of the attributions across color channels for visualization.
    # The attribution mask shape is a grayscale image with height and width
    # equal to the original image.
    pos_attribution_mask, neg_attribution_mask = process_attributions(attributions, rescale_power)
    
    combined_masks = np.zeros_like(attributions)
    combined_masks[:, :, 0] = pos_attribution_mask
    combined_masks[:, :, 2] = neg_attribution_mask

    # Plotting
    plt.figure(figsize=(19,16))

    plt.suptitle("Attribution Visualization for the {} Class\n\n\n" \
                 "".format(class_label), fontsize=16, fontweight="bold")
    
    plt.subplot(2,3,1)
    plt.title('\nBaseline image', fontweight="bold")
    plt.imshow(baseline)
    plt.axis('off')
    
    plt.subplot(2,3,2)
    plt.title('\nPositive Attribution Mask', fontweight="bold")
    plt.imshow(pos_attribution_mask, cmap=pos_cmap)
    plt.axis('off')

    plt.subplot(2,3,3)
    plt.title('\nNegative Attribution Mask', fontweight="bold")
    plt.imshow(neg_attribution_mask, cmap=neg_cmap)
    plt.axis('off')

    plt.subplot(2,3,4)
    plt.title('Original image', fontweight="bold")
    plt.imshow(image/255)
    plt.axis('off')

    plt.subplot(2,3,5)
    plt.title('Full Attribution mask', fontweight="bold")
    plt.imshow(combined_masks)
    plt.axis('off')

    plt.subplot(2,3,6)
    plt.title('Overlay', fontweight="bold")
    # plt.imshow(np.sum(combined_masks, axis=2), cmap="gray")
    plt.imshow(combined_masks)
    plt.imshow(image/255, alpha=overlay_alpha)
    plt.axis('off')
    
    if save_fig:
        plt.savefig(f"/kaggle/working/{c}.jpg" ,dpi=400)
    
    plt.tight_layout()
    plt.show()
    
    return combined_masks

@tf.function(experimental_relax_shapes=False)
def integrated_gradients(model, 
                         baseline,
                         image,
                         target_class_idx,
                         m_steps=64,
                         batch_size=8):
    

    # Step 0. Format Check
    if tf.math.reduce_max(image)<=1.:
        image *= 255.
    if tf.math.reduce_max(baseline)<=1.:
        baseline *= 255.
    
    # Step 1. Generate alphas
    alphas = tf.linspace(start=tf.constant(0, dtype=tf.float32), 
                         stop=tf.constant(1, dtype=tf.float32), 
                         num=m_steps)

    # Accumulate gradients across batches
    integrated_gradients = 0.0

    # Batch alpha images
    ds = tf.data.Dataset.from_tensor_slices(alphas).batch(batch_size)

    for batch in ds:
        # Step 2. Generate interpolated images
        batch_interpolated_inputs = interpolate_images(baseline=baseline/255,
                                                       image=image/255,
                                                       alphas=batch)

        # Step 3. Compute gradients between model outputs and interpolated inputs
        batch_gradients = compute_gradients(model,
                                            images=batch_interpolated_inputs,
                                            target_class_idx=target_class_idx)

        # Step 4. Average integral approximation. Summing integrated gradients across batches.
        integrated_gradients += integral_approximation(gradients=batch_gradients)

    # Step 5. Scale integrated gradients with respect to input
    scaled_integrated_gradients = (image - baseline) * integrated_gradients
    return scaled_integrated_gradients

def get_labels_from_id(target_id, df):
    return [int(x) for x in df[df.ID==target_id].Label.str.split("|").to_list()[0]]

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="model">5&nbsp;&nbsp;LOAD AND DEMONSTRATE MODEL PREDICTION</a>

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: uppercase; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.1 LOAD THE TRAINED MODEL</h3>

---

In [ ]:
with strategy.scope():
    load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
    enet = tf.keras.models.load_model(MODEL_DIR, options=load_locally) # loading in Tensorflow's "SavedModel" format

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: uppercase; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.2 SHOW MODEL PREDICTIONS</h3>

---

- Grab a random batch of single label IDs
- Get images and labels for batch from each label-count dataframe
- Get predictions for batch at a given confidence threshold
- Visualize

In [ ]:
DEMO_BATCH_SIZE = 4

for i in range(1, 6):
    if i==5:
        DEMO_BATCH_SIZE=len(dfs_by_label_cnt[i])
    print(f"\n... {DEMO_BATCH_SIZE} PREDICTIONS FOR IMAGES WITH {i} LABELS ...\n")
    RNDM_SNGL_LBL_IDS = dfs_by_label_cnt[i].ID.sample(DEMO_BATCH_SIZE).sort_index().to_list()
    print(RNDM_SNGL_LBL_IDS)
    img_batch, lbl_batch = load_batch_of_images(dfs_by_label_cnt[i], RNDM_SNGL_LBL_IDS, TRAIN_IMG_DIR, tpu_style=True)
    pred_batch = get_pred(enet, img_batch, conf_thresh=0.225, drop_yellow=True)
    plot_batch_of_images(img_batch, lbl_batch, pred_batch, n_cols=4, labels_as_strs=False, boost_green=True)

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: uppercase; letter-spacing: 2px; color: navy; background-color: #ffffff;">5.3 SHOW INTEGRATED GRADIENTS ON A DIFFICULT, MULTI-LABEL IMAGE</h3>

---

- Calculate IG For The Same Image Monitoring Each GT Label Respectively
- Preprocess These Attributions To Improve Visability (Rescale Between 0-1 and Raise to Fractional Power)
- Visualize

In [ ]:
IG_IMG_ID = "8e998b50-bbb0-11e8-b2ba-ac1f6b6435d0"
BASELINE = tf.zeros(shape=(512,512,3), dtype=tf.float32)
IG_IMG = tf.cast(load_image(IG_IMG_ID, TRAIN_IMG_DIR, tpu_style=True), tf.float32)
IG_LBLS = get_labels_from_id(IG_IMG_ID, train_df)
CMAP="magma"

IG_LBL_MAP = {
    l:integrated_gradients(enet, 
                           BASELINE,
                           image=IG_IMG[..., :-1], 
                           target_class_idx=tf.constant(l)) \
    for l in tqdm(IG_LBLS)}

In [ ]:
attr_img = np.zeros((512,512,len(IG_LBL_MAP)))
plt.figure(figsize=(20,16))
for i, (cls,ig_attr) in enumerate(IG_LBL_MAP.items()):
    attr = process_attributions(ig_attr, rescale_power=tf.constant(1.4, dtype=tf.float32), grayscale=True)
    plt.subplot(2,int(np.ceil((len(IG_LBL_MAP)+1)/2)),i+1)
    plt.imshow(attr, cmap=CMAP)
    plt.title(f"{INT_2_STR[cls]} Attribution Map", fontweight="bold")
    plt.axis(False)

# Show the ground truth image    
plt.subplot(2,int(np.ceil((len(IG_LBL_MAP)+1)/2)),len(IG_LBL_MAP)+1)
plt.imshow(convert_rgby_to_rgb(IG_IMG.numpy(), boost_green=True))
plt.title("Image With Green Channel Boosted", fontweight="bold")
plt.axis(False)

plt.tight_layout()
plt.savefig("demonstration_ig.png", dpi=600)
plt.show()